In [84]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import warnings
warnings.filterwarnings("ignore")
import os
import re
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk import FreqDist
from nltk.tokenize import word_tokenize

In [2]:
for dirname,_,filenames in os.walk('movie-review-sentiment-analysis'):
    for filename in filenames:
        print(os.path.join(dirname,filename))

movie-review-sentiment-analysis\sampleSubmission.csv
movie-review-sentiment-analysis\test.tsv
movie-review-sentiment-analysis\train.tsv


In [3]:
df_train=pd.read_csv(r"movie-review-sentiment-analysis\train.tsv",sep='\t')
df_test=pd.read_csv(r"movie-review-sentiment-analysis\test.tsv",sep='\t')
df_train.head()

,PhraseId,SentenceId,Phrase,Sentiment
0,1,1,A series of escapades demonstrating the adage ...,1
1,2,1,A series of escapades demonstrating the adage ...,2
2,3,1,A series,2
3,4,1,A,2
4,5,1,series,2


In [4]:
'''
import csv
with open(r"movie-review-sentiment-analysis\train.tsv") as tsvfile:
    tsvreader = csv.reader(tsvfile, delimiter="\t")
    for line in tsvreader:
        print(line)
'''

'\nimport csv\nwith open(r"movie-review-sentiment-analysis\train.tsv") as tsvfile:\n    tsvreader = csv.reader(tsvfile, delimiter="\t")\n    for line in tsvreader:\n        print(line)\n'

In [5]:
df_train.isna().sum()

PhraseId      0
SentenceId    0
Phrase        0
Sentiment     0
dtype: int64

In [6]:
lem=WordNetLemmatizer()
stop=set(stopwords.words("english"))
df_train.shape 

(156060, 4)

In [7]:
'''
for i in (df_train['Phrase'][:100]):
    #review=i.split()
    #review=i.lower()
    #review=review.split()
    review=re.sub('[^a-zA-Z]',' ',i)
    review=review.split()
    #review=review.lower()
    #review=re.sub('[^a-zA-Z]',' ',text)
    review=[lem.lemmatize(j) for j in review if not j in stopwords.words('english')]
    review=' '.join(review)
'''

"\nfor i in (df_train['Phrase'][:100]):\n    #review=i.split()\n    #review=i.lower()\n    #review=review.split()\n    review=re.sub('[^a-zA-Z]',' ',i)\n    review=review.split()\n    #review=review.lower()\n    #review=re.sub('[^a-zA-Z]',' ',text)\n    review=[lem.lemmatize(j) for j in review if not j in stopwords.words('english')]\n    review=' '.join(review)\n"

In [8]:
# removing the shit and then stemming it
def prep(text):
    review=re.sub('[^a-zA-Z]',' ',text)
    review=review.lower()
    review=review.split()
    review=[ lem.lemmatize(word) for word in review if not word in stopwords.words('english')]
    review=' '.join(review)
    return review

In [9]:
df_train['cleaned_text']=df_train['Phrase'].apply(lambda x: prep(x))

In [10]:
df_test['cleaned_text']=df_test['Phrase'].apply(lambda x: prep(x))

In [11]:
df_train['cleaned_text'][:10]

0    series escapade demonstrating adage good goose...
1       series escapade demonstrating adage good goose
2                                               series
3                                                     
4                                               series
5              escapade demonstrating adage good goose
6                                                     
7              escapade demonstrating adage good goose
8                                             escapade
9                       demonstrating adage good goose
Name: cleaned_text, dtype: object

In [29]:
df_train.Sentiment.value_counts()/len(df_train.Sentiment)

2    0.509945
3    0.210989
1    0.174760
4    0.058990
0    0.045316
Name: Sentiment, dtype: float64

In [13]:
X=df_train.cleaned_text
y=df_train.Sentiment

In [27]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score,confusion_matrix

In [15]:
vec=TfidfVectorizer()

In [20]:
vec_train=vec.fit_transform(X)

In [21]:
vec_train

<156060x13606 sparse matrix of type '<class 'numpy.float64'>'
	with 618229 stored elements in Compressed Sparse Row format>

In [17]:
vec_test=vec.fit_transform(df_test['cleaned_text'])

In [22]:
X_train,X_test,y_train,y_test=train_test_split(vec_train,y,test_size=.2)

In [23]:
log=LogisticRegression()

In [25]:
log.fit(X_train,y_train)
pred_log=log.predict(X_test)

In [28]:
print("accuracy_score",accuracy_score(y_test,pred_log))
print("confusion_matrix\n",confusion_matrix(y_test,pred_log))

accuracy_score 0.6185121107266436
confusion_matrix
 [[  189   695   518    41     2]
 [  114  1674  3372   238     3]
 [   30   688 14265   916    31]
 [    4   148  3389  2778   221]
 [    0    23   413  1061   399]]


# Finding number of unique words in train set

In [86]:
all_words=' '.join(X)
all_words1=word_tokenize(all_words)
dist=FreqDist(all_words1)
num_unique_word=len(dist)
num_unique_word

13625

# Finding max length of a review in train set

In [93]:
r_len=[]
for text in X:
    word=word_tokenize(text)
    l=len(word)
    r_len.append(l)
    
MAX_REVIEW_LEN=np.max(r_len)
MAX_REVIEW_LEN

30

In [96]:
max_features = num_unique_word
max_words = MAX_REVIEW_LEN
batch_size = 128
epochs = 3
num_classes=5

In [ ]:
tokenizer = Tokenizer(num_words=max_features)